# Обновленный код функции integrate

In [9]:
import math
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
from functools import partial
import timeit

def integrate(f, a, b, *, n_iter=1000):
    h = (b - a) / n_iter
    result = 0.5 * (f(a) + f(b))
    for i in range(1, n_iter):
        x = a + i * h
        result += f(x)
    return result * h

def integrate_async(f, a, b, *, n_jobs=2, n_iter=1000, executor_class):
    with executor_class(max_workers=n_jobs) as executor:
        spawn = partial(executor.submit, integrate, f, n_iter=n_iter // n_jobs)
        step = (b - a) / n_jobs
        fs = [spawn(a + i * step, a + (i + 1) * step) for i in range(n_jobs)]
        return sum(f.result() for f in as_completed(fs))

def measure_time(func, *args, **kwargs):
    try:
        return timeit.timeit(lambda: func(*args, **kwargs), number=10) * 100  # в миллисекундах
    except Exception as e:
        print(f"Ошибка при измерении времени: {e}")
        return float('inf')

if __name__ == '__main__':
    # Параметры интегрирования
    f = math.atan
    a, b = 0, math.pi / 2
    n_iter = 10**6

    # Измерение времени для последовательного решения
    time_sequential = measure_time(integrate, f, a, b, n_iter=n_iter)

    # Измерение времени для асинхронных решений
    for n_jobs in [2, 4, 6]:
        time_thread = measure_time(integrate_async, f, a, b, n_jobs=n_jobs, n_iter=n_iter, executor_class=ThreadPoolExecutor)
        time_process = measure_time(integrate_async, f, a, b, n_jobs=n_jobs, n_iter=n_iter, executor_class=ProcessPoolExecutor)
        
        print(f"\nКоличество потоков/процессов: {n_jobs}")
        print(f"Последовательное решение: {time_sequential:.2f} мс")
        print(f"Многопоточное решение: {time_thread:.2f} мс")
        print(f"Многопроцессное решение: {time_process:.2f} мс")
        if time_thread != float('inf'):
            print(f"Ускорение (потоки): {time_sequential / time_thread:.2f}x")
        if time_process != float('inf'):
            print(f"Ускорение (процессы): {time_sequential / time_process:.2f}x")

    # Проверка точности результатов
    result_sequential = integrate(f, a, b, n_iter=n_iter)
    result_thread = integrate_async(f, a, b, n_jobs=4, n_iter=n_iter, executor_class=ThreadPoolExecutor)
    try:
        result_process = integrate_async(f, a, b, n_jobs=4, n_iter=n_iter, executor_class=ProcessPoolExecutor)
    except Exception as e:
        print(f"Ошибка при многопроцессном вычислении: {e}")
        result_process = None

    print("\nПроверка точности:")
    print(f"Последовательное решение: {result_sequential:.6f}")
    print(f"Многопоточное решение: {result_thread:.6f}")
    if result_process is not None:
        print(f"Многопроцессное решение: {result_process:.6f}")
    print(f"Точное значение: {math.pi * math.log(2) / 2:.6f}")

Ошибка при измерении времени: A process in the process pool was terminated abruptly while the future was running or pending.

Количество потоков/процессов: 2
Последовательное решение: 158.99 мс
Многопоточное решение: 115.30 мс
Многопроцессное решение: inf мс
Ускорение (потоки): 1.38x
Ошибка при измерении времени: A process in the process pool was terminated abruptly while the future was running or pending.

Количество потоков/процессов: 4
Последовательное решение: 158.99 мс
Многопоточное решение: 115.91 мс
Многопроцессное решение: inf мс
Ускорение (потоки): 1.37x
Ошибка при измерении времени: A process in the process pool was terminated abruptly while the future was running or pending.

Количество потоков/процессов: 6
Последовательное решение: 158.99 мс
Многопоточное решение: 119.86 мс
Многопроцессное решение: inf мс
Ускорение (потоки): 1.33x
Ошибка при многопроцессном вычислении: A process in the process pool was terminated abruptly while the future was running or pending.

Проверка т

# Программа, симулирующую работу банка с использованием потоков и объектов типа Lock


In [1]:
import threading
import time
import random

class BankAccount:
    def __init__(self, balance=0):
        self.balance = balance
        self.lock = threading.Lock()

    def deposit(self, amount):
        with self.lock:
            self.balance += amount
            print(f"Внесено {amount}. Новый баланс: {self.balance}")

    def withdraw(self, amount):
        with self.lock:
            if self.balance >= amount:
                self.balance -= amount
                print(f"Снято {amount}. Новый баланс: {self.balance}")
                return True
            else:
                print(f"Недостаточно средств для снятия {amount}. Текущий баланс: {self.balance}")
                return False

def customer(account, name):
    for _ in range(5):  # Каждый клиент совершает 5 операций
        action = random.choice(['deposit', 'withdraw'])
        amount = random.randint(50, 200)
        
        if action == 'deposit':
            print(f"{name} пытается внести {amount}")
            account.deposit(amount)
        else:
            print(f"{name} пытается снять {amount}")
            account.withdraw(amount)
        
        time.sleep(random.random())  # Случайная пауза между операциями

# Создаем банковский счет
account = BankAccount(1000)  # Начальный баланс 1000

# Создаем потоки для клиентов
threads = []
for i in range(3):  # 3 клиента
    thread = threading.Thread(target=customer, args=(account, f"Клиент {i+1}"))
    threads.append(thread)
    thread.start()

# Ждем завершения всех потоков
for thread in threads:
    thread.join()

print(f"Итоговый баланс: {account.balance}")

Клиент 1 пытается снять 167
Снято 167. Новый баланс: 833
Клиент 2 пытается внести 158
Внесено 158. Новый баланс: 991
Клиент 3 пытается внести 134
Внесено 134. Новый баланс: 1125
Клиент 2 пытается внести 156
Внесено 156. Новый баланс: 1281
Клиент 1 пытается снять 129
Снято 129. Новый баланс: 1152
Клиент 3 пытается внести 183
Внесено 183. Новый баланс: 1335
Клиент 2 пытается снять 132
Снято 132. Новый баланс: 1203
Клиент 1 пытается снять 126
Снято 126. Новый баланс: 1077
Клиент 2 пытается внести 109
Внесено 109. Новый баланс: 1186
Клиент 3 пытается снять 74
Снято 74. Новый баланс: 1112
Клиент 2 пытается снять 196
Снято 196. Новый баланс: 916
Клиент 3 пытается снять 79
Снято 79. Новый баланс: 837
Клиент 1 пытается внести 137
Внесено 137. Новый баланс: 974
Клиент 3 пытается внести 158
Внесено 158. Новый баланс: 1132
Клиент 1 пытается внести 141
Внесено 141. Новый баланс: 1273
Итоговый баланс: 1273


# Программа, которая асинхронно скачивает изображения с использованием объектов типа Future и ограничивает количество одновременных загрузок с помощью семафора

In [2]:
import os
import requests
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.parse import urlparse

# Создаем семафор для ограничения количества одновременных загрузок
MAX_CONCURRENT_DOWNLOADS = 3
semaphore = threading.Semaphore(MAX_CONCURRENT_DOWNLOADS)

def download_image(url):
    with semaphore:
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                # Получаем имя файла из URL
                filename = os.path.basename(urlparse(url).path)
                # Сохраняем изображение
                with open(filename, 'wb') as f:
                    f.write(response.content)
                print(f"Успешно загружено: {filename}")
                return filename
            else:
                print(f"Ошибка при загрузке {url}: статус {response.status_code}")
                return None
        except Exception as e:
            print(f"Ошибка при загрузке {url}: {str(e)}")
            return None

def main():
    image_urls = [
        "https://i.pinimg.com/originals/30/a0/2b/30a02b9d7517d5b27803c8180d8123c6.jpg",
        "https://i.pinimg.com/originals/7e/1b/fd/7e1bfd1191112533fe9872ef47398823.jpg",
        "https://cdn.akamai.steamstatic.com/steamcommunity/public/images/clans/30297037/46db5ab0b1643199f65eef34933fba16c022b0d5.jpg",
        "https://i.pinimg.com/736x/be/04/b7/be04b74f78b7ffb889e6c5de67f5cc16.jpg",
        "https://avatars.mds.yandex.net/i?id=a50dea91092171a296bf5d79c7332e54_l-4836347-images-thumbs&n=13",
    ]

    # Создаем директорию для сохранения изображений
    os.makedirs("concurrency_downloaded_images", exist_ok=True)
    os.chdir("concurrency_downloaded_images")

    # Используем ThreadPoolExecutor для асинхронной загрузки
    with ThreadPoolExecutor(max_workers=MAX_CONCURRENT_DOWNLOADS) as executor:
        # Создаем Future объекты для каждой загрузки
        future_to_url = {executor.submit(download_image, url): url for url in image_urls}

        # Обрабатываем результаты по мере их завершения
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                filename = future.result()
                if filename:
                    print(f"Загрузка {url} завершена")
                else:
                    print(f"Не удалось загрузить {url}")
            except Exception as e:
                print(f"Ошибка при обработке {url}: {str(e)}")

if __name__ == "__main__":
    main()

Успешно загружено: 7e1bfd1191112533fe9872ef47398823.jpg
Загрузка https://i.pinimg.com/originals/7e/1b/fd/7e1bfd1191112533fe9872ef47398823.jpg завершена
Успешно загружено: 30a02b9d7517d5b27803c8180d8123c6.jpg
Загрузка https://i.pinimg.com/originals/30/a0/2b/30a02b9d7517d5b27803c8180d8123c6.jpg завершена
Успешно загружено: 46db5ab0b1643199f65eef34933fba16c022b0d5.jpg
Загрузка https://cdn.akamai.steamstatic.com/steamcommunity/public/images/clans/30297037/46db5ab0b1643199f65eef34933fba16c022b0d5.jpg завершена
Успешно загружено: be04b74f78b7ffb889e6c5de67f5cc16.jpg
Загрузка https://i.pinimg.com/736x/be/04/b7/be04b74f78b7ffb889e6c5de67f5cc16.jpg завершена
Успешно загружено: i
Загрузка https://avatars.mds.yandex.net/i?id=a50dea91092171a296bf5d79c7332e54_l-4836347-images-thumbs&n=13 завершена


# Программа с двумя потоками, где один поток будет записывать данные в файл, а другой - считывать их.

In [3]:
import time
from concurrent.futures import ThreadPoolExecutor, Future

# Глобальная переменная для управления работой потоков
running = True

def writer(filename):
    counter = 0
    while running:
        with open(filename, 'a', encoding='utf-8') as f:
            f.write(f"Запись {counter}\n")
        print(f"Записано: Запись {counter}")
        counter += 1
        time.sleep(1)  # Пауза между записями

def reader(filename, future):
    last_position = 0
    while running:
        with open(filename, 'r', encoding='utf-8') as f:
            f.seek(last_position)
            new_data = f.read()
            if new_data:
                print(f"Прочитано: {new_data.strip()}")
                last_position = f.tell()
        time.sleep(0.5)  # Пауза между чтениями
    future.set_result("Чтение завершено")

def main():
    global running
    filename = "io_threads.txt"
    
    # Создаем пустой файл или очищаем существующий
    open(filename, 'w').close()

    with ThreadPoolExecutor(max_workers=2) as executor:
        # Запускаем поток записи
        write_future = executor.submit(writer, filename)
        
        # Создаем Future объект для потока чтения
        read_future = Future()
        
        # Запускаем поток чтения
        executor.submit(reader, filename, read_future)

        try:
            time.sleep(10)
        finally:
            # Останавливаем потоки
            running = False
            
            # Ждем завершения потока чтения
            result = read_future.result()
            print(result)

if __name__ == "__main__":
    main()

Записано: Запись 0
Прочитано: Запись 0
Записано: Запись 1
Прочитано: Запись 1
Записано: Запись 2
Прочитано: Запись 2
Записано: Запись 3
Прочитано: Запись 3
Записано: Запись 4
Прочитано: Запись 4
Записано: Запись 5
Прочитано: Запись 5
Записано: Запись 6
Прочитано: Запись 6
Записано: Запись 7
Прочитано: Запись 7
Записано: Запись 8
Прочитано: Запись 8
Записано: Запись 9
Прочитано: Запись 9
Чтение завершено


# Программа, которая создает три потока

In [4]:
import threading
import time

# Создаем объект Event
event = threading.Event()

# Флаг для управления работой потоков
running = True

def set_event():
    global running
    while running:
        time.sleep(1)
        event.set()
        print("Event set")
        event.clear()

def wait_for_event():
    global running
    while running:
        if event.wait(timeout=1.1):  # Ждем чуть дольше, чем интервал установки события
            print("Event occurred")

def check_event():
    global running
    while running and not event.is_set():
        print("Event did not occur")
        time.sleep(1)
    if running:
        print("Event occurred, third thread stopping")

# Создаем потоки
thread1 = threading.Thread(target=set_event)
thread2 = threading.Thread(target=wait_for_event)
thread3 = threading.Thread(target=check_event)

# Запускаем потоки
thread1.start()
thread2.start()
thread3.start()

# Даем потокам поработать некоторое время
time.sleep(10)

# Останавливаем потоки
running = False

# Ждем завершения всех потоков
thread1.join()
thread2.join()
thread3.join()

print("All threads have finished")

Event did not occur
Event set
Event occurred
Event did not occur
Event setEvent occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occurred
Event occur

# Программа с классом Queue, который использует RLock для обеспечения потокобезопасности

In [5]:
import threading
import time
import random

class Queue:
    def __init__(self):
        self.items = []
        self.lock = threading.RLock()

    def enqueue(self, item):
        with self.lock:
            self.items.append(item)
            print(f"Добавлено: {item}")

    def dequeue(self):
        with self.lock:
            if not self.is_empty():
                item = self.items.pop(0)
                print(f"Удалено: {item}")
                return item
            else:
                print("Очередь пуста")
                return None

    def is_empty(self):
        with self.lock:
            return len(self.items) == 0

    def size(self):
        with self.lock:
            return len(self.items)

def producer(queue, name):
    for i in range(5):
        item = f"Item-{name}-{i}"
        queue.enqueue(item)
        time.sleep(random.uniform(0.1, 0.5))

def consumer(queue, name):
    for i in range(5):
        item = queue.dequeue()
        if item:
            print(f"Потребитель {name} получил: {item}")
        time.sleep(random.uniform(0.1, 0.5))

def main():
    queue = Queue()

    # Создаем потоки производителей и потребителей
    producers = [threading.Thread(target=producer, args=(queue, f"P{i}")) for i in range(3)]
    consumers = [threading.Thread(target=consumer, args=(queue, f"C{i}")) for i in range(2)]

    # Запускаем все потоки
    all_threads = producers + consumers
    for thread in all_threads:
        thread.start()

    # Ждем завершения всех потоков
    for thread in all_threads:
        thread.join()

    print("Все операции завершены")
    print(f"Конечный размер очереди: {queue.size()}")

if __name__ == "__main__":
    main()

Добавлено: Item-P0-0
Добавлено: Item-P1-0
Добавлено: Item-P2-0
Удалено: Item-P0-0
Потребитель C0 получил: Item-P0-0
Удалено: Item-P1-0
Потребитель C1 получил: Item-P1-0
Добавлено: Item-P2-1
Удалено: Item-P2-0
Потребитель C1 получил: Item-P2-0
Добавлено: Item-P0-1
Удалено: Item-P2-1
Потребитель C1 получил: Item-P2-1
Удалено: Item-P0-1
Потребитель C0 получил: Item-P0-1
Добавлено: Item-P0-2
Добавлено: Item-P1-1
Добавлено: Item-P2-2
Удалено: Item-P0-2
Потребитель C1 получил: Item-P0-2
Добавлено: Item-P1-2
Удалено: Item-P1-1
Потребитель C0 получил: Item-P1-1
Добавлено: Item-P0-3
Удалено: Item-P2-2
Потребитель C1 получил: Item-P2-2
Добавлено: Item-P2-3
Добавлено: Item-P0-4
Добавлено: Item-P1-3
Удалено: Item-P1-2
Потребитель C0 получил: Item-P1-2
Добавлено: Item-P1-4
Удалено: Item-P0-3
Потребитель C0 получил: Item-P0-3
Добавлено: Item-P2-4
Все операции завершены
Конечный размер очереди: 5


# Программа с двумя потоками, представляющими сервер и клиент, использующая threading.Barrier

In [6]:
import threading
import time
import random

# Создаем барьер для двух потоков (сервер и клиент)
barrier = threading.Barrier(2)

# Флаг для управления работой потоков
running = True

def server():
    global running
    print("Сервер: Запуск...")
    time.sleep(random.uniform(1, 3))  # Имитация времени запуска сервера
    print("Сервер: Готов к работе")
    
    while running:
        try:
            barrier.wait(timeout=2)  # Ожидание готовности клиента с таймаутом
            if not running:
                break
            print("Сервер: Обработка запроса клиента")
            time.sleep(random.uniform(0.5, 1.5))  # Имитация обработки запроса
            print("Сервер: Запрос обработан")
            
            barrier.wait(timeout=2)  # Сигнал клиенту, что ответ готов
        except threading.BrokenBarrierError:
            # Барьер сломан, вероятно, клиент завершил работу
            break
    
    print("Сервер: Завершение работы")

def client():
    global running
    print("Клиент: Ожидание готовности сервера...")
    
    for _ in range(5):  # Клиент отправит 5 запросов
        barrier.wait()  # Ожидание готовности сервера
        print("Клиент: Отправка запроса")
        time.sleep(random.uniform(0.1, 0.5))  # Имитация формирования запроса
        
        barrier.wait()  # Ожидание ответа от сервера
        print("Клиент: Получен ответ от сервера")
        time.sleep(random.uniform(0.5, 1))  # Имитация обработки ответа
    
    running = False  # Сигнал для завершения работы сервера
    try:
        barrier.wait()  # Последний вызов, чтобы разблокировать сервер
    except threading.BrokenBarrierError:
        pass  # Игнорируем ошибку, так как мы намеренно ломаем барьер
    print("Клиент: Завершение работы")

# Создаем и запускаем потоки
server_thread = threading.Thread(target=server)
client_thread = threading.Thread(target=client)

server_thread.start()
client_thread.start()

# Ожидаем завершения обоих потоков
server_thread.join()
client_thread.join()

print("Программа завершена")

Сервер: Запуск...
Клиент: Ожидание готовности сервера...
Сервер: Готов к работе
Сервер: Обработка запроса клиента
Клиент: Отправка запроса
Сервер: Запрос обработан
Клиент: Получен ответ от сервера
Клиент: Отправка запроса
Сервер: Обработка запроса клиента
Сервер: Запрос обработан
Клиент: Получен ответ от сервера
Клиент: Отправка запросаСервер: Обработка запроса клиента

Сервер: Запрос обработан
Клиент: Получен ответ от сервера
Клиент: Отправка запросаСервер: Обработка запроса клиента

Сервер: Запрос обработан
Клиент: Получен ответ от сервера
Клиент: Отправка запросаСервер: Обработка запроса клиента

Сервер: Запрос обработан
Клиент: Получен ответ от сервера
Клиент: Завершение работыСервер: Завершение работы

Программа завершена


# Программа для параллельного поиска файла в директории

In [7]:
import os
import threading
import fnmatch
from concurrent.futures import ThreadPoolExecutor, as_completed

class FileSearcher:
    def __init__(self, root_dir, pattern):
        self.root_dir = root_dir
        self.pattern = pattern
        self.found_event = threading.Event()
        self.found_file = None

    def search_in_directory(self, directory):
        for root, _, files in os.walk(directory):
            if self.found_event.is_set():
                return  # Прекращаем поиск, если файл уже найден

            for filename in files:
                if self.found_event.is_set():
                    return  # Прекращаем поиск, если файл уже найден

                if fnmatch.fnmatch(filename, self.pattern):
                    full_path = os.path.join(root, filename)
                    self.found_file = full_path
                    self.found_event.set()  # Сигнализируем о нахождении файла
                    return

    def parallel_search(self, num_threads=4):
        directories = [os.path.join(self.root_dir, d) for d in os.listdir(self.root_dir) 
                       if os.path.isdir(os.path.join(self.root_dir, d))]
        
        with ThreadPoolExecutor(max_workers=num_threads) as executor:
            futures = [executor.submit(self.search_in_directory, dir) for dir in directories]
            
            for future in as_completed(futures):
                if self.found_event.is_set():
                    break  # Прерываем выполнение, если файл найден

        if self.found_file:
            print(f"Файл найден: {self.found_file}")
        else:
            print("Файл не найден")


root_directory = "E:/Repositories/rgpu_7sem"
file_pattern = "*.txt"
searcher = FileSearcher(root_directory, file_pattern)
searcher.parallel_search()

Файл найден: E:/Repositories/rgpu_7sem\Программирование\lr1\io_threads.txt
